In [1]:
print("Hello, World!")


Hello, World!


Set up OpenAPI client to use OpenAPI embeddings:

In [2]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')


In [3]:
import qdrant_client
from qdrant_client.http.models import Distance, VectorParams
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Qdrant


Initialize new persistent collection in Qdrant:

In [4]:
collection_name = "test"

embeddings = OpenAIEmbeddings()
client = qdrant_client.QdrantClient(
    path="/tmp/qdrant_store", prefer_grpc=True
)

partial_embeddings = embeddings.embed_documents(["Hello, World!"])
vector_size = len(partial_embeddings[0])

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=vector_size,
        distance=Distance.COSINE,
    ),
)

qdrant = Qdrant(
    client=client, collection_name=collection_name,
    embedding_function=embeddings.embed_query
)


Now add some texts to Qdrant to see if it works:

In [5]:
qdrant.add_texts(["Hello, World!"])


['65a8e27d8879283831b664bd8b7f0ad4']

In [6]:
query = "What was said to the World?"
found_docs = qdrant.similarity_search(query)


In [7]:
print(found_docs)


[Document(page_content='Hello, World!', metadata={})]


Clean up:
1. Find the document that we've just inserted

In [8]:
from qdrant_client.http.models import Filter, FieldCondition, MatchText

points, _ = client.scroll(
    collection_name=collection_name,
    scroll_filter=Filter(
        must=[
            FieldCondition(
                key="page_content",
                match=MatchText(text="Hello, World!"),
            ),
        ]
    ),
)
points


[Record(id='65a8e27d8879283831b664bd8b7f0ad4', payload={'page_content': 'Hello, World!', 'metadata': None}, vector=None)]

2. Delete all the documents that we found

In [9]:
from qdrant_client.http.models import PointIdsList

client.delete(collection_name=collection_name, points_selector=PointIdsList(
    points=[point.id for point in points],
),
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)